In [1]:
import pandas as pd
import numpy as np
import time
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import requests
import json
project_id = "perceptive-ivy-290216"

In [2]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.sprint_lap_time`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pd.read_gbq(query=query, project_id=project_id, dialect='standard')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_4970/3254455452.py:10: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_bq2 = pd.read_gbq(query=query, project_id=project_id, dialect='standard')


In [33]:
df_bq=df_bq2[(df_bq2["GP"]=='Austrian Grand Prix')&(df_bq2["Year"]==2024)]
GP=df_bq
GP["LapNumber"]=GP["LapNumber"].astype(int)
GP=GP.sort_values(['Driver','LapNumber'])
GP["DriverNumber"]=GP["DriverNumber"].astype(int)
year=GP['Year'].iloc[0]
gp=GP['GP'].iloc[0]
GP.head()

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_4970/248577147.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP
5824,0 days 00:49:51.803000,ALB,23,NaT,1,1.0,0 days 00:48:39.573000,NaT,NaT,0 days 00:00:32.286000,0 days 00:00:22.094000,NaT,0 days 00:49:29.821000,0 days 00:49:51.878000,305.0,215.0,272.0,300.0,False,MEDIUM,7.0,False,Williams,0 days 00:48:29.599000,2024-06-29 10:06:04.325,1,20.0,False,,False,False,2024,Austrian Grand Prix
5498,0 days 00:51:03.420000,ALB,23,0 days 00:01:11.617000,2,1.0,NaT,NaT,0 days 00:00:17.767000,0 days 00:00:31.833000,0 days 00:00:22.017000,0 days 00:50:09.605000,0 days 00:50:41.438000,0 days 00:51:03.455000,311.0,225.0,282.0,307.0,True,MEDIUM,8.0,False,Williams,0 days 00:49:51.803000,2024-06-29 10:07:26.529,1,20.0,False,,False,True,2024,Austrian Grand Prix
5499,0 days 00:52:14.296000,ALB,23,0 days 00:01:10.876000,3,1.0,NaT,NaT,0 days 00:00:17.118000,0 days 00:00:31.733000,0 days 00:00:22.025000,0 days 00:51:20.573000,0 days 00:51:52.306000,0 days 00:52:14.331000,322.0,227.0,284.0,315.0,True,MEDIUM,9.0,False,Williams,0 days 00:51:03.420000,2024-06-29 10:08:38.146,1,20.0,False,,False,True,2024,Austrian Grand Prix
5500,0 days 00:53:25.758000,ALB,23,0 days 00:01:11.462000,4,1.0,NaT,NaT,0 days 00:00:17.218000,0 days 00:00:32.108000,0 days 00:00:22.136000,0 days 00:52:31.549000,0 days 00:53:03.657000,0 days 00:53:25.793000,320.0,227.0,280.0,307.0,False,MEDIUM,10.0,False,Williams,0 days 00:52:14.296000,2024-06-29 10:09:49.022,1,20.0,False,,False,True,2024,Austrian Grand Prix
5501,0 days 00:54:36.801000,ALB,23,0 days 00:01:11.043000,5,1.0,NaT,NaT,0 days 00:00:17.204000,0 days 00:00:31.994000,0 days 00:00:21.845000,0 days 00:53:42.997000,0 days 00:54:14.991000,0 days 00:54:36.836000,303.0,227.0,276.0,302.0,False,MEDIUM,11.0,False,Williams,0 days 00:53:25.758000,2024-06-29 10:11:00.484,1,20.0,False,,False,True,2024,Austrian Grand Prix


In [34]:
#Convert LapTime to Time Delta for plotting on Y Axis
GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()

# Fastest Driver in each lap

In [35]:
GP['s1_TD']= pd.to_timedelta(GP["Sector1Time"])
GP['s2_TD']= pd.to_timedelta(GP["Sector2Time"])
GP['s3_TD']= pd.to_timedelta(GP["Sector3Time"])

GP.loc[:, "Time S1 (s)"] = GP["s1_TD"].dt.total_seconds()
GP.loc[:, "Time S2 (s)"] = GP["s2_TD"].dt.total_seconds()
GP.loc[:, "Time S3 (s)"] = GP["s3_TD"].dt.total_seconds()

In [36]:
GP_Rk=GP[GP["LapTime"]!="NaT"]
GP_Rk["LAP_RK"] = GP_Rk.groupby("LapNumber")["LapTime (s)"].rank(method="dense", ascending=True)
GP_Rk["S1_RK"] = GP_Rk.groupby("LapNumber")["Time S1 (s)"].rank(method="dense", ascending=True)
GP_Rk["S2_RK"] = GP_Rk.groupby("LapNumber")["Time S2 (s)"].rank(method="dense", ascending=True)
GP_Rk["S3_RK"] = GP_Rk.groupby("LapNumber")["Time S3 (s)"].rank(method="dense", ascending=True)
GP_Rk["Race_RK"] = GP_Rk.groupby("GP")["LapTime (s)"].rank(method="dense", ascending=True)


In [37]:
GP_Sectors=GP_Rk[["Driver","LapNumber","Time S1 (s)","Time S2 (s)","Time S3 (s)","LapTime","LapTime (s)","Compound","Year","GP","LAP_RK","S1_RK","S2_RK","S3_RK"]]
GP_Sectors=GP_Sectors.fillna(0)
GP_Sectors.sort_values(by=["LapNumber","S1_RK"]).head()

,Driver,LapNumber,Time S1 (s),Time S2 (s),Time S3 (s),LapTime,LapTime (s),Compound,Year,GP,LAP_RK,S1_RK,S2_RK,S3_RK
5824,ALB,1,0.0,32.286,22.094,0,0.0,MEDIUM,2024,Austrian Grand Prix,0.0,0.0,8.0,12.0
5817,ALO,1,0.0,33.032,22.654,0,0.0,MEDIUM,2024,Austrian Grand Prix,0.0,0.0,14.0,18.0
5822,BOT,1,0.0,33.565,22.607,0,0.0,MEDIUM,2024,Austrian Grand Prix,0.0,0.0,18.0,16.0
5813,GAS,1,0.0,33.259,22.102,0,0.0,MEDIUM,2024,Austrian Grand Prix,0.0,0.0,16.0,13.0
5810,HAM,1,0.0,32.008,21.920,0,0.0,MEDIUM,2024,Austrian Grand Prix,0.0,0.0,5.0,7.0


In [38]:
def driver_color(driver):
    if driver =="VER":
        return "#3860C6"
    elif driver =="PER":
        return '#3860C6'
    elif driver =="NOR":
        return '#FF8000'
    elif driver =="PIA":
        return '#FF8000'
    elif driver =="LEC":
        return '#E80020'
    elif driver =="SAI":
        return '#E80020'
    elif driver =="HAM":
        return '#27F4D2'
    elif driver =="RUS":
        return '#27F4D2'
    elif driver =="ALO":
        return '#229971'
    elif driver =="STR":
        return '#229971'
    elif driver =="GAS":
        return '#0093cc'
    elif driver =="OCO":
        return '#0093cc'
    elif driver =="MAG":
        return '#B6BABD'
    elif driver =="HUL":
        return '#B6BABD'
    elif driver =="ZHO":
        return '#52e252'
    elif driver =="BOT":
        return '#52e252'
    elif driver =="ALB":
        return '#64C4FF'
    elif driver =="SAR":
        return '#64C4FF'
    elif driver =="TSU":
        return '#6692FF'
    elif driver =="RIC":
        return '#6692FF'
    else:
        return '#3860C6'

In [39]:
#get a combined view for fastest drivers in the entire lap, S1, S2, and S3
Fastest_Lap=GP_Sectors[GP_Sectors["LAP_RK"]==1][["GP","Year","LapNumber","LapTime","LapTime (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_Lap.columns=["GP","Year","LapNumber","LapTime","LapTime (s)","Lap Compound","Fastest Lap"]
Fastest_sector1=GP_Sectors[GP_Sectors["S1_RK"]==1][["LapNumber","Time S1 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector1.columns=["LapNumber","Time S1 (s)","S1 Compound","Fastest S1"]
Fastest_sector2=GP_Sectors[GP_Sectors["S2_RK"]==1][["LapNumber","Time S2 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector2.columns=["LapNumber","Time S2 (s)","S2 Compound","Fastest S2"]
Fastest_sector3=GP_Sectors[GP_Sectors["S3_RK"]==1][["LapNumber","Time S3 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector3.columns=["LapNumber","Time S3 (s)","S3 Compound","Fastest S3"]
Fast_Laps=Fastest_Lap.merge(Fastest_sector1,on=['LapNumber']).merge(Fastest_sector2,on=['LapNumber']).merge(Fastest_sector3,on=['LapNumber'])
Fast_Laps=Fast_Laps[["GP","Year","LapNumber","LapTime","LapTime (s)","Fastest Lap","Lap Compound","Time S1 (s)","Fastest S1","S1 Compound","Time S2 (s)","Fastest S2","S2 Compound","Time S3 (s)","Fastest S3","S3 Compound"]].sort_values(by=["LapNumber"])
Fast_Laps=Fast_Laps.reset_index(drop=True)
Fast_Laps.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound
0,Austrian Grand Prix,2024,2,00:01:08.935000,68.935,NOR,MEDIUM,17.171,NOR,MEDIUM,30.794,PIA,MEDIUM,20.647,NOR,MEDIUM
1,Austrian Grand Prix,2024,3,00:01:09.261000,69.261,SAI,MEDIUM,16.969,HAM,MEDIUM,30.990,SAI,MEDIUM,20.533,VER,MEDIUM
2,Austrian Grand Prix,2024,4,00:01:09.047000,69.047,PIA,MEDIUM,17.005,RUS,MEDIUM,30.959,PIA,MEDIUM,20.573,VER,MEDIUM
3,Austrian Grand Prix,2024,5,00:01:09.025000,69.025,PIA,MEDIUM,16.965,SAR,MEDIUM,31.038,PIA,MEDIUM,20.481,VER,MEDIUM
4,Austrian Grand Prix,2024,6,00:01:09.059000,69.059,VER,MEDIUM,16.864,HAM,MEDIUM,30.861,VER,MEDIUM,20.638,VER,MEDIUM


In [40]:
Fast_Laps['color'] = Fast_Laps['Fastest Lap'].apply(driver_color)
Fast_Laps.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound,color
0,Austrian Grand Prix,2024,2,00:01:08.935000,68.935,NOR,MEDIUM,17.171,NOR,MEDIUM,30.794,PIA,MEDIUM,20.647,NOR,MEDIUM,#FF8000
1,Austrian Grand Prix,2024,3,00:01:09.261000,69.261,SAI,MEDIUM,16.969,HAM,MEDIUM,30.990,SAI,MEDIUM,20.533,VER,MEDIUM,#E80020
2,Austrian Grand Prix,2024,4,00:01:09.047000,69.047,PIA,MEDIUM,17.005,RUS,MEDIUM,30.959,PIA,MEDIUM,20.573,VER,MEDIUM,#FF8000
3,Austrian Grand Prix,2024,5,00:01:09.025000,69.025,PIA,MEDIUM,16.965,SAR,MEDIUM,31.038,PIA,MEDIUM,20.481,VER,MEDIUM,#FF8000
4,Austrian Grand Prix,2024,6,00:01:09.059000,69.059,VER,MEDIUM,16.864,HAM,MEDIUM,30.861,VER,MEDIUM,20.638,VER,MEDIUM,#3860C6


In [41]:
import plotly.graph_objects as go
# colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

fig = go.Figure(go.Table
 (header={"values": Fast_Laps.columns},
  cells=dict(
    values=Fast_Laps.T.values,
    fill_color=[Fast_Laps.color],
    align='center', font=dict(color='white', size=14)
    ))
 )

fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": c,
                    "method": "update",
                    "args": [
                        {
                            "cells": {
                                "values": Fast_Laps.T.values
                                if c == "All"
                                else Fast_Laps.loc[Fast_Laps["LapNumber"].eq(c)].T.values
                            }
                        }
                    ],
                }
                for c in ["All"] + Fast_Laps["LapNumber"].unique().tolist()
            ]
        }
    ]
)
fig.update_layout(
    title="Sprint Race Fastest Lap and Sector Times For Ordered By Lap Number",
        title_x=0.5,
        # template="plotly_dark",

    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="Black"
    ),
    height=1000,
    title_font_family="PT Sans Narrow"
)

fig.show()

In [42]:
fig.write_html("{} {} Sprint Race Fastest Lap Times and Sectors Ordered by Lap Number.html".format(year,gp))

In [43]:
LapTime_ordered=Fast_Laps.sort_values(by=['LapTime (s)'])
LapTime_ordered.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound,color
0,Austrian Grand Prix,2024,2,00:01:08.935000,68.935,NOR,MEDIUM,17.171,NOR,MEDIUM,30.794,PIA,MEDIUM,20.647,NOR,MEDIUM,#FF8000
6,Austrian Grand Prix,2024,8,00:01:09.013000,69.013,VER,MEDIUM,17.032,RUS,MEDIUM,30.793,VER,MEDIUM,20.555,VER,MEDIUM,#3860C6
3,Austrian Grand Prix,2024,5,00:01:09.025000,69.025,PIA,MEDIUM,16.965,SAR,MEDIUM,31.038,PIA,MEDIUM,20.481,VER,MEDIUM,#FF8000
2,Austrian Grand Prix,2024,4,00:01:09.047000,69.047,PIA,MEDIUM,17.005,RUS,MEDIUM,30.959,PIA,MEDIUM,20.573,VER,MEDIUM,#FF8000
4,Austrian Grand Prix,2024,6,00:01:09.059000,69.059,VER,MEDIUM,16.864,HAM,MEDIUM,30.861,VER,MEDIUM,20.638,VER,MEDIUM,#3860C6


In [44]:
import plotly.graph_objects as go
# colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

fig2 = go.Figure(go.Table
 (header={"values": LapTime_ordered.columns},
  cells=dict(
    values=LapTime_ordered.T.values,
    fill_color=[LapTime_ordered.color],
    align='center', font=dict(color='white', size=14)
    ))
 )

fig2.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": c,
                    "method": "update",
                    "args": [
                        {
                            "cells": {
                                "values": LapTime_ordered.T.values
                                if c == "All"
                                else LapTime_ordered.loc[LapTime_ordered["LapNumber"].eq(c)].T.values
                            }
                        }
                    ],
                }
                for c in ["All"] + LapTime_ordered["LapNumber"].unique().tolist()
            ]
        }
    ]
)
fig2.update_layout(
    title="Fastest Lap and Sector Times Ordered By Lap Times",
        title_x=0.5,
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="Black"
    ),
    height=1000,
    title_font_family="PT Sans Narrow"
)

fig2.show()

In [45]:
fig2.write_html("{} {} Sprint Race Fastest Lap Times and Sectors Ordered by Lap Times.html".format(year,gp))

In [46]:
#get a shortened view for fastest drivers in the entire lap, S1, S2, and S3

Fast_Laps_short=Fast_Laps[["GP","Year","LapNumber","Fastest Lap","Fastest S1","Fastest S2","Fastest S3","color"]].sort_values(by=["LapNumber"])
Fast_Laps_short=Fast_Laps_short.reset_index(drop=True)
Fast_Laps_short.head()

,GP,Year,LapNumber,Fastest Lap,Fastest S1,Fastest S2,Fastest S3,color
0,Austrian Grand Prix,2024,2,NOR,NOR,PIA,NOR,#FF8000
1,Austrian Grand Prix,2024,3,SAI,HAM,SAI,VER,#E80020
2,Austrian Grand Prix,2024,4,PIA,RUS,PIA,VER,#FF8000
3,Austrian Grand Prix,2024,5,PIA,SAR,PIA,VER,#FF8000
4,Austrian Grand Prix,2024,6,VER,HAM,VER,VER,#3860C6


In [47]:
import plotly.graph_objects as go
# colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

fig = go.Figure(go.Table
 (header={"values": Fast_Laps_short.columns},
  cells=dict(
    values=Fast_Laps_short.T.values,
    height=50,
    fill_color=[Fast_Laps_short.color],
    align='center', font=dict(color='white', size=20)
    ))
 )

fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": c,
                    "method": "update",
                    "args": [
                        {
                            "cells": {
                                "values": Fast_Laps_short.T.values
                                if c == "All"
                                else Fast_Laps_short.loc[Fast_Laps_short["LapNumber"].eq(c)].T.values
                            }
                        }
                    ],
                }
                for c in ["All"] + Fast_Laps_short["LapNumber"].unique().tolist()
            ]
        }
    ]
)
fig.update_layout(
    title="Fastest Lap and Sector Times",
        title_x=0.5,
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=20,
        color="Black"
    ),
    height=2500,
    title_font_family="PT Sans Narrow"
)

fig.show()

In [48]:
#get a view for fastest lap in the entire race
Fastest_Lap_Race=GP_Rk[GP_Rk["Race_RK"]==1][["Year","GP","Driver","LapNumber","Time S1 (s)","Time S2 (s)","Time S3 (s)","LapTime","LapTime (s)","Compound"]].reset_index(drop=True)
Fastest_Lap_Race

,Year,GP,Driver,LapNumber,Time S1 (s),Time S2 (s),Time S3 (s),LapTime,LapTime (s),Compound
0,2024,Austrian Grand Prix,NOR,2,17.171,31.117,20.647,00:01:08.935000,68.935,MEDIUM


In [49]:
#Multiple plotly filters code
import plotly.graph_objects as go

df = pd.DataFrame(
    {
        "Date": ["2020-01-27", "2020-02-27", "2020-03-27"],
        "A_item": [2, 8, 0],
        "B_item": [1, 7, 10],
        "C_item": [9, 2, 9],
        "Channel_type": ["Channel_1", "Channel_1", "Channel_2"],
    }
)

fig = go.Figure(go.Table(header={"values": df.columns}, cells={"values": df.T.values}))
fig.update_layout(
    updatemenus=[
        {
            "y": 1 - (i / 5),
            "buttons": [
                {
                    "label": c,
                    "method": "restyle",
                    "args": [
                        {
                            "cells": {
                                "values": df.T.values
                                if c == "All"
                                else df.loc[df[menu].eq(c)].T.values
                            }
                        }
                    ],
                }
                for c in ["All"] + df[menu].unique().tolist()
            ],
        }
        for i, menu in enumerate(["Channel_type", "Date"])
    ]
)

In [50]:
#Single Plotly filter code
import plotly.graph_objects as go

df = pd.DataFrame(
    {
        "Date": ["2020-01-27", "2020-02-27", "2020-03-27"],
        "A_item": [2, 8, 0],
        "B_item": [1, 7, 10],
        "C_item": [9, 2, 9],
        "Channel_type": ["Channel_1", "Channel_1", "Channel_2"],
    }
)

fig = go.Figure(go.Table(header={"values": df.columns}, cells={"values": df.T.values}))
fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": c,
                    "method": "update",
                    "args": [
                        {
                            "cells": {
                                "values": df.T.values
                                if c == "All"
                                else df.loc[df["Channel_type"].eq(c)].T.values
                            }
                        }
                    ],
                }
                for c in ["All"] + df["Channel_type"].unique().tolist()
            ]
        }
    ]
)